<a href="https://colab.research.google.com/github/myllanes/Introduction-to-Deep-Learning/blob/main/HW6_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
#Michael Yllanes
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import time
from tqdm import tqdm

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Hyperparameters
image_size = 32       # CIFAR image size
patch_size = 8        # 4x4 patches
num_classes = 100     # 100 classes for CIFAR-100
embed_dim = 512       # Embedding dimension
num_heads = 4         # Attention heads
num_layers = 4        # Transformer layers
mlp_ratio = 4         # MLP hidden dim = embed_dim * mlp_ratio
dropout = 0.1         # Dropout rate
batch_size = 128      # Batch size
num_epochs = 20      # CIFAR-100
learning_rate = 0.001
weight_decay = 0.03   # Weight decay

# CIFAR-100
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  # CIFAR-100 stats
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# CIFAR-100 dataset
train_dataset = torchvision.datasets.CIFAR100(
    root='./data', train=True, download=True, transform=train_transform)
test_dataset = torchvision.datasets.CIFAR100(
    root='./data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

class PatchEmbedding(nn.Module):
    """Converts images into patch embeddings with convolutional projection"""
    def __init__(self, img_size=32, patch_size=4, in_chans=3, embed_dim=256):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.num_patches = (img_size // patch_size) ** 2

        self.proj = nn.Conv2d(
            in_chans, embed_dim,
            kernel_size=patch_size, stride=patch_size
        )

    def forward(self, x):
        x = self.proj(x)  # [B, embed_dim, H', W']
        x = x.flatten(2)  # [B, embed_dim, num_patches]
        x = x.transpose(1, 2)  # [B, num_patches, embed_dim]
        return x

class TransformerBlock(nn.Module):
    """Transformer encoder block with pre-norm architecture"""
    def __init__(self, embed_dim, num_heads, mlp_ratio, dropout):
        super().__init__()
        self.norm1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads, dropout=dropout, batch_first=True)
        self.norm2 = nn.LayerNorm(embed_dim)

        mlp_hidden_dim = int(embed_dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(embed_dim, mlp_hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden_dim, embed_dim),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        # Self-attention with residual connection
        x = x + self.attn(self.norm1(x), self.norm1(x), self.norm1(x))[0]
        # MLP with residual connection
        x = x + self.mlp(self.norm2(x))
        return x

class VisionTransformer(nn.Module):
    """Complete ViT model for CIFAR-100 classification"""
    def __init__(self, img_size=32, patch_size=4, num_classes=100,
                 embed_dim=256, num_heads=2, num_layers=4,
                 mlp_ratio=2, dropout=0.1):
        super().__init__()

        self.patch_embed = PatchEmbedding(img_size, patch_size, 3, embed_dim)
        num_patches = self.patch_embed.num_patches

        # Learnable parameters
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        self.dropout = nn.Dropout(dropout)

        # Transformer encoder
        self.blocks = nn.Sequential(*[
            TransformerBlock(embed_dim, num_heads, mlp_ratio, dropout)
            for _ in range(num_layers)
        ])

        # Classification head
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

        # Initialize weights
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        nn.init.trunc_normal_(self.pos_embed, std=0.02)
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.trunc_normal_(m.weight, std=0.02)
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.LayerNorm):
            nn.init.zeros_(m.bias)
            nn.init.ones_(m.weight)

    def forward(self, x):
        B = x.shape[0]

        # Patch embedding
        x = self.patch_embed(x)

        # Add class token
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)

        # Add position embeddings
        x = x + self.pos_embed
        x = self.dropout(x)

        # Transformer blocks
        x = self.blocks(x)

        # Classification
        x = self.norm(x)
        return self.head(x[:, 0])

# Initialize model
model = VisionTransformer(
    img_size=image_size,
    patch_size=patch_size,
    num_classes=num_classes,
    embed_dim=embed_dim,
    num_heads=num_heads,
    num_layers=num_layers,
    mlp_ratio=mlp_ratio,
    dropout=dropout
).to(device)

# Loss function with label smoothing
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Optimizer with weight decay
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay
)

# Learning rate scheduler
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

def train_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss, total_correct, total_samples = 0, 0, 0

    for images, labels in tqdm(loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()

        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        # Metrics
        _, predicted = torch.max(outputs.data, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        total_loss += loss.item() * labels.size(0)

    avg_loss = total_loss / total_samples
    accuracy = 100 * total_correct / total_samples
    return avg_loss, accuracy

def evaluate(model, loader, criterion):
    model.eval()
    total_loss, total_correct, total_samples = 0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            total_correct += (predicted == labels).sum().item()
            total_loss += loss.item() * labels.size(0)

    avg_loss = total_loss / total_samples
    accuracy = 100 * total_correct / total_samples
    return avg_loss, accuracy

# Training loop with validation and timing
best_acc = 0.0
total_training_time = 0.0  # total training time

print("\nStarting training...")
overall_start_time = time.time()

for epoch in range(num_epochs):
    epoch_start_time = time.time()

    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = evaluate(model, test_loader, criterion)

    scheduler.step()

    # Calculate epoch time
    epoch_time = time.time() - epoch_start_time
    total_training_time += epoch_time

    # Print metrics
    print(f"Time: {epoch_time:.2f}s")
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

# Calculate total training time
total_time = time.time() - overall_start_time

# Print final results and timing information
print(f"Total Training Time: {total_time:.2f} seconds")
print(f"Average Time per Epoch: {total_time/num_epochs:.2f} seconds")

Using device: cuda

Starting training...

Epoch 1/20


Evaluating: 100%|██████████| 79/79 [00:00<00:00, 79.46it/s]


Time: 14.16s
Train Loss: 4.3289 | Train Acc: 5.21%
Val Loss: 4.2980 | Val Acc: 5.81%

Epoch 2/20


Evaluating: 100%|██████████| 79/79 [00:00<00:00, 83.85it/s]


Time: 14.00s
Train Loss: 4.2578 | Train Acc: 5.98%
Val Loss: 4.2228 | Val Acc: 6.58%

Epoch 3/20


...

Time: 13.73s
Train Loss: 3.9043 | Train Acc: 12.30%
Val Loss: 3.9324 | Val Acc: 12.27%

Epoch 19/20


Evaluating: 100%|██████████| 79/79 [00:00<00:00, 82.02it/s]


Time: 13.74s
Train Loss: 3.8951 | Train Acc: 12.37%
Val Loss: 3.9318 | Val Acc: 12.04%

Epoch 20/20


Evaluating: 100%|██████████| 79/79 [00:00<00:00, 84.24it/s]

Time: 13.70s
Train Loss: 3.8950 | Train Acc: 12.54%
Val Loss: 3.9301 | Val Acc: 12.13%
Total Training Time: 275.14 seconds
Average Time per Epoch: 13.76 seconds


In [8]:
!pip install torchinfo
from torchinfo import summary
import time

# Using torchinfo
# Model summary
summary(model, input_size=(batch_size, 3, 32, 32),
       verbose=1, col_names=["input_size", "output_size", "num_params", "mult_adds"])

# Timing
starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
repetitions = 100
timings = []

for _ in range(10):
    _ = model(torch.randn(batch_size, 3, 32, 32).to(device))

# Measurement
with torch.no_grad():
    for _ in range(repetitions):
        inputs = torch.randn(batch_size, 3, 32, 32).to(device)
        starter.record()
        _ = model(inputs)
        ender.record()
        torch.cuda.synchronize()
        timings.append(starter.elapsed_time(ender))

avg_time = sum(timings) / repetitions
print(f"Average forward pass time: {avg_time:.2f}ms")
print(f"Throughput: {1000/(avg_time)*batch_size:.0f} samples/sec")

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Mult-Adds
VisionTransformer                        [128, 3, 32, 32]          [128, 100]                9,216                     --
├─PatchEmbedding: 1-1                    [128, 3, 32, 32]          [128, 16, 512]            --                        --
│    └─Conv2d: 2-1                       [128, 3, 32, 32]          [128, 512, 4, 4]          98,816                    202,375,168
├─Dropout: 1-2                           [128, 17, 512]            [128, 17, 512]            --                        --
├─Sequential: 1-3                        [128, 17, 512]            [128, 17, 512]            --                        --
│    └─TransformerBlock: 2-2             [128, 17, 512]            [128, 17, 512]            --                        --
│    │    └─LayerNorm: 3-1               [128, 17, 512]            [128, 17, 512]            1,024                     131,072
│  